In [4]:
import pandas as pd
import numpy as np
from pprint import pprint
from nanoHUB.application import Application
from nanoHUB.configuration import ClusteringConfiguration
from nanoHUB.pipeline.geddes.data import get_default_s3_client
from nanoHUB.dataaccess.lake import S3FileMapper
import os
cwd = os.getcwd()

# logger.debug('Testing')
application = Application.get_instance()
nanohub_db = application.new_db_engine('nanohub')

salesforce = application.new_salesforce_engine()

s3_client = get_default_s3_client(application)
raw_mapper = S3FileMapper(s3_client, ClusteringConfiguration().bucket_name_raw)
processed_mapper = S3FileMapper(s3_client, ClusteringConfiguration().bucket_name_processed)

derived_data_file_path = 'derived_data_for_users.csv'



Obtained Salesforce access token ...... True


In [6]:
query = '''
SELECT Id, nanoHUB_user_ID__c FROM Contact
WHERE Id IN (SELECT Contact__c FROM ContactToolClusterAssociation__c)
'''
contacts_df = salesforce.query_data(query)
display(contacts_df)

[Success] Bulk job creation successful. Job ID = 7505w00000dtFcvAAE
{"id":"7505w00000dtFcvAAE","operation":"query","object":"Contact","createdById":"0055w00000DM5bOAAT","createdDate":"2022-02-18T20:54:17.000+0000","systemModstamp":"2022-02-18T20:54:17.000+0000","state":"UploadComplete","concurrencyMode":"Parallel","contentType":"CSV","apiVersion":47.0,"jobType":"V2Query","lineEnding":"LF","columnDelimiter":"COMMA","retries":0,"totalProcessingTime":0}
{"id":"7505w00000dtFcvAAE","operation":"query","object":"Contact","createdById":"0055w00000DM5bOAAT","createdDate":"2022-02-18T20:54:17.000+0000","systemModstamp":"2022-02-18T20:54:24.000+0000","state":"JobComplete","concurrencyMode":"Parallel","contentType":"CSV","apiVersion":47.0,"jobType":"V2Query","lineEnding":"LF","columnDelimiter":"COMMA","numberRecordsProcessed":66359,"retries":0,"totalProcessingTime":4936}
[Success] Bulk job completed successfully.


,Id,nanoHUB_user_ID__c
0,0035w000031VvlBAAS,67859.0
1,0035w000034JJ0aAAG,7997.0
2,0035w000034IW7DAAW,150267.0
3,0035w000034JK4sAAG,38451.0
4,0035w000034Is9NAAS,276743.0
...,...,...
66354,0035w000034JrvPAAS,130080.0
66355,0035w000034JrHoAAK,129332.0
66356,0035w000034JrwqAAC,130206.0
66357,0035w000034Jr4CAAS,129044.0


In [8]:
sql_query = 'select id, uid, author from jos_citations'
jos_citations = pd.read_sql_query(sql_query, nanohub_db)
display(jos_citations.head(2))
display(jos_citations.shape) 

,id,uid,author
0,10000001,5568,"Weber, Bent; Mahapatra, Suddhasatta; Ryu, Hoon..."
1,10000002,5568,"Andrawis, Robert; Bermeo, Jose; Charles, James..."


(4203, 3)

In [10]:
t1_authors = jos_citations['author'].to_list()
t1_authors2 = [] 
for i,j in enumerate(t1_authors):
    try:
        holder = j.split(';')
        if len(holder) > 1: 
            for k,l in enumerate(holder):
                if l[0] == ' ':
                    holder[k] = l[1:]
        t1_authors2.append(holder)
    except:
        garb = True
        
t1_authors32 = [item for sublist in t1_authors2 for item in sublist]
t1_authors3 = set(t1_authors32)
print(len(t1_authors3))

6860


In [11]:
## extract all researchers with nh ids
rw_ids = []
need_search = []
for i,j in enumerate(t1_authors3):
    if '{{' in j:
        rw_ids.append(i)
    else:
        need_search.append(i)
        
rw_ids2 = np.array(list(t1_authors3))[rw_ids]
for i,j in enumerate(rw_ids2):
    begin = j.index('{{')
    rw_ids2[i] = j[begin+2:-2]
print(len(rw_ids2))

2079


In [ ]:
sql_query = "select id,name,username from jos_users where id in "+str(tuple(rw_ids2))
rw_researchers = pd.read_sql_query(sql_query,nanohub_db)
display(rw_researchers.head(2))

In [ ]:
ns2 = np.array(list(t1_authors3))[need_search]
ns2 = ns2[1:]
tuple(ns2)[:5]

In [ ]:
sql_query = "select id,name,username from jos_users where name in "+str(tuple(ns2))
ns2_researchers = pd.read_sql_query(sql_query,nanohub_db)
display(ns2_researchers.head(2))

In [ ]:
researcher_usernames = rw_researchers['username'].to_list() + ns2_researchers['username'].to_list()
print(len(researcher_usernames))

In [ ]:
researcher_track = []
today = datetime.datetime.today()

try:
    user_breakdown_df = pd.read_csv(cwd+'/nh_user_breakdown.csv')
    print(user_breakdown_df)
    cache_flag = True
except:
    cache_flag = False

In [ ]:
start_year_base = 1999
print(user_breakdown_df['year_month'].to_list()[-1][:4])

In [ ]:
## computer has memory limits, so split toolstart into branches
# this filters and finds the research usage in nanohub
start_year = deepcopy(start_year_base)
while start_year < today.year+1:
    start_year += 1
    end_date = r"'"+str(start_year)+r"-01-01'"
    start_date = r"'"+str(start_year-2)+r"-01-01'"
    
    for i in range(1,13):
        if i < 10:
            start_date = r"'"+str(start_year-1)+r"-0"+str(i)+r"-01'"
            end_date = r"'"+str(start_year)+r"-0"+str(i)+r"-01'"
        else:
            start_date = r"'"+str(start_year-1)+r"-"+str(i)+r"-01'"
            end_date = r"'"+str(start_year)+r"-"+str(i)+r"-01'"

        sql_query_researcher = "select user from toolstart where user in " + str(tuple(researcher_usernames))\
            +" and datetime <= "+end_date+" and datetime >= "+start_date
        researcher_users = pd.read_sql_query(sql_query_researcher, nanohub_metrics_db)

        researcher_track.append(researcher_users.drop_duplicates().shape[0])

    print('start year: '+ str(start_year))

In [ ]:
researcher_ids = rw_researchers['id'].to_list() + ns2_researchers['id'].to_list()
print(len(researcher_ids))

In [ ]:
raise

In [2]:
all_users_df = processed_mapper.read(derived_data_file_path)
all_users_df['id'] = all_users_df.id.astype(int)
#all_users_df['is_researcher'] = False
display(all_users_df.tail())

[WARNING] [warnings - py.warnings]: /tmp/ipykernel_2483/2458013441.py:1: DtypeWarning: Columns (6) have mixed types.Specify dtype option on import or set low_memory=False.
  all_users_df = processed_mapper.read(derived_data_file_path)
 [warnings._showwarnmsg:109]


,id,name,username,email,registerDate,lastvisitDate,first_citation_at,number_citations,profile_key,has_researcher_profile,number_simulations,uid
272686,356675,Ra Abd,eyad2020,rabab.r3ab@gmail.com,2022-02-12 00:28:49,2022-02-12 00:29:39,NaN,0.0,[],False,0.0,NaN
272687,356676,Pede John Villadares Garzon,john1973,pedejohngarzon@gmail.com,2022-02-12 00:45:10,2022-02-12 00:45:11,NaN,0.0,[],False,0.0,NaN
272688,356677,Busra Ozdemir,-193184,-193184@invalid,2022-02-12 00:58:08,2022-02-12 00:58:08,NaN,0.0,[],False,0.0,NaN
272689,356679,Mahdi Salari,mahdisalari82020,mahdisalari82020@gmail.com,2022-02-12 02:18:29,2022-02-12 02:18:30,NaN,0.0,[],False,0.0,NaN
272690,356680,王鸣雁,-193187,-193187@invalid,2022-02-12 02:27:35,2022-02-12 02:27:36,NaN,0.0,[],False,0.0,NaN


In [3]:
sql_query = "select * from jos_citations_authors;"
authors_df = pd.read_sql_query(sql_query, nanohub_db)

display(authors_df)
raise

,id,cid,author,authorid,uidNumber,ordering,givenName,middleName,surname,organization,...,ipCITY,ipLATITUDE,ipLONGITUDE,in_network,orcid,research_id,gscholar_id,scopus_id,researchgate_id,notes
0,189134,10001576,Woody Gilbertson,0,100100,4,Woody,,Gilbertson,Purdue University,...,,NaN,NaN,0,None,None,None,None,None,None
1,189135,10001576,Hesameddin Ilatikhameneh,0,42735,5,Hesameddin,,Ilatikhameneh,Purdue University,...,,NaN,NaN,0,None,None,None,None,None,None
2,189132,10001576,Daniel F Mejia,0,52349,2,Daniel,F,Mejia,Purdue University,...,,NaN,NaN,0,None,None,None,None,None,None
3,189133,10001576,James Charles,0,65635,3,James,,Charles,Purdue University,...,,NaN,NaN,0,None,None,None,None,None,None
4,189131,10001576,Prasad Sarangapani,0,72949,1,Prasad,,Sarangapani,Purdue University,...,,NaN,NaN,0,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16884,17997619,1280,Yaohua Tan,3015,30369,0,Yaohua,P.,Tan,Purdue University,...,,0.0,0.0,0,,,,,,
16885,17997617,256,Xingwu Liu,628,0,2,Xingwu,,Liu,Chinese Academy Of Sciences,...,BEIJING,0.0,0.0,-1,,,,,,
16886,17997616,256,Lijuan Xiao,627,0,1,Lijuan,,Xiao,Chinese Academy Of Sciences,...,BEIJING,0.0,0.0,-1,,,,,,
16887,17997618,1024,J. Ramani,2334,0,0,J.,,Ramani,Ohio University,...,,0.0,0.0,0,,,,,,


RuntimeError: No active exception to reraise

In [ ]:
# Basic tool information
sql_query = '''
SELECT c.uid, c.created
    FROM jos_citations as c 
    LEFT JOIN jos_citations_types AS t 
    ON c.type = t.id \
'''

citations_df = pd.read_sql_query(sql_query, nanohub_db)
citations_df = citations_df.dropna()
citations_df['number_citations'] = citations_df.groupby(['uid'])["created"].transform("count")
citations_df = citations_df.drop_duplicates(subset = ['uid'])
citations_df['uid'] = citations_df.uid.astype(int)
citations_df = citations_df.rename({'created': 'first_citation_at'}, axis=1) 

display(len(citations_df))

In [ ]:
#unique_authors = citations_df.uid.unique().tolist()
#unique_authors = list(map(int, unique_authors))
#display(unique_authors)


In [ ]:
all_users_df = all_users_df.merge(citations_df, how='left', left_on='id', right_on='uid')
all_users_df = all_users_df.drop(columns='uid')

all_users_df['number_citations'].fillna(0, inplace=True)
condition = all_users_df["number_citations"] > 0
#all_users_df["is_researcher"] = np.where(condition, True, all_users_df['is_researcher'])



In [ ]:
#display(all_users_df.loc[all_users_df['is_researcher'].notna()])

In [ ]:
display(all_users_df)

In [ ]:
# get user profile details
sql_query = '''
SELECT user_id, profile_key FROM jos_user_profiles
WHERE profile_key in ('researcherid', 'googlescholar', 'orcid', 'researchgateid', 'scopusid')
'''
profile_df = pd.read_sql_query(sql_query, nanohub_db)



display(len(profile_df))
display(profile_df.head())
display(profile_df.tail())

In [ ]:
display(profile_df.profile_key.unique())

In [ ]:
profile_df = profile_df.groupby(['user_id'], as_index=False)['profile_key'].agg(list)

In [ ]:
display(profile_df)

In [ ]:
profile_df['has_researcher_profile'] = True
profile_df['user_id'] = profile_df['user_id'].astype(int)
display(profile_df[profile_df["user_id"] == 2862])

all_users_df = all_users_df.merge(profile_df, how='left', left_on='id', right_on='user_id')
display(all_users_df[all_users_df["id"] == 2862])

all_users_df['has_researcher_profile'].fillna(False,inplace=True)
all_users_df['profile_key'] = all_users_df['profile_key'].apply(lambda d: d if isinstance(d, list) else [])
all_users_df = all_users_df.drop(columns='user_id')

In [ ]:
display(len(all_users_df[all_users_df["profile_key"].str.len() == 0]))
display(len(all_users_df[all_users_df["profile_key"].str.len() == 1]))
display(all_users_df[all_users_df["id"] == 2579])
display(all_users_df[all_users_df['id'] == 998])

In [ ]:
processed_mapper.save_as_csv(all_users_df, derived_data_file_path, index=None)

test_df = processed_mapper.read(derived_data_file_path)
display(test_df)

In [ ]:
display("# of users with citations > 1 = %d" % len(test_df[test_df['number_citations'] > 1]))